In [85]:
import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OrdinalEncoder 
from sklearn.model_selection import train_test_split

In [ ]:
#!pip install pycaret==3.0.4
#!pip install xgboost==2.0.3
#!pip install lightgbm==3.3.5

In [86]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
test.head(3)

,Id,carat,cut,color,clarity,depth,table,x,y,z
0,1,0.34,Ideal,G,VVS2,61.1,57.0,4.52,4.48,2.75
1,2,0.71,Premium,E,VS2,62.7,58.0,5.74,5.68,3.58
2,3,0.44,Very Good,I,VS1,62.8,56.0,4.83,4.88,3.05


In [101]:
df=pd.read_csv('train.csv')
df=df.drop(labels=['Id','depth','table'],axis=1)
val_df = pd.read_csv('test.csv')
val_df_proc = val_df.drop(labels=['depth','table'],axis=1)

X = df.drop(labels=['price'],axis=1)
Y = df[['price']]

In [88]:
categorical_cols = ['cut', 'color', 'clarity']
numerical_cols = ['carat', 'x', 'y', 'z']
##
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [90]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])

In [102]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=30)
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())
val_df_proc = pd.DataFrame(preprocessor.fit_transform(val_df_proc),columns=preprocessor.get_feature_names_out())
df_pycaret = pd.DataFrame(preprocessor.fit_transform(X),columns=preprocessor.get_feature_names_out())
df_pycaret.head()

,num_pipeline__carat,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.553529,0.747755,0.752947,0.750367,0.982064,1.413690,-1.239768
1,1.503722,1.470153,1.466739,1.371635,0.085364,0.238432,1.185614
2,-1.009010,-1.160802,-1.153397,-1.169914,0.982064,-0.349198,-0.027077
3,-0.565586,-0.482997,-0.448311,-0.492168,0.982064,0.238432,-0.027077
4,-0.206625,0.025357,-0.013072,-0.054456,0.085364,0.826061,1.185614


In [92]:
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,KFold

models={'XGboost':XGBRegressor()}
cvv=KFold(n_splits=6,shuffle=True,random_state=21)

for name,model in models.items():
    cross_scores = cross_val_score(model,X_train,y_train,cv=cvv,scoring='neg_mean_squared_error')
    rmse=np.sqrt(-cross_scores)
    print(f'{name}')
    print(f'scores:{rmse}')
    print(f'mean score:{np.mean(rmse)}')
    print(f'std score:{np.std(rmse)}')
    print('*'*10)


model=XGBRegressor()
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],  
    'max_depth': [3, 4, 5,7],            
    'n_estimators': [30, 50, 100]
}

grid=GridSearchCV(model,param_grid,cv=cvv)
grid.fit(X_train,y_train)


y_pred_train = grid.predict(X_train) 
y_pred_test = grid.predict(X_test)

rmse_train= np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_val= np.sqrt(mean_squared_error(y_test, y_pred_test))

print('Train RMSE = ' + str(rmse_train))
print('Test RMSE = ' + str(rmse_val))

XGboost
scores:[591.44496649 555.32160375 556.76048423 552.07301374 579.71450304
 549.56541387]
mean score:564.1466641872825
std score:15.697017538666248
**********
Train RMSE = 423.63047859503865
Test RMSE = 542.2443159833929


In [93]:
val_df_proc["Id"] = val_df["Id"]
submit_CSV = pd.DataFrame()
price = grid.predict(val_df_proc.drop('Id',axis= 1))
submit_CSV['Id']= val_df_proc['Id']
submit_CSV['price']= price
submit_CSV.to_csv('Late_Sub_5_XGBR.csv',index= None)

# Pycaret

In [94]:
from sklearn.metrics._scorer import _SCORERS
from pycaret.regression import *

df_pycaret['price'] = df[['price']]
caret_df = setup(data = df_pycaret, target = 'price', session_id=55)
best = compare_models(n_select = 10)
best

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,12:27:11
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


Processing:   0%|          | 0/86 [00:00<?, ?it/s]

[LGBMRegressor(n_jobs=-1, random_state=55),
 RandomForestRegressor(n_jobs=-1, random_state=55),
 ExtraTreesRegressor(n_jobs=-1, random_state=55),
 GradientBoostingRegressor(random_state=55),
 KNeighborsRegressor(n_jobs=-1),
 DecisionTreeRegressor(random_state=55),
 AdaBoostRegressor(random_state=55),
 Lasso(random_state=55),
 BayesianRidge(),
 Ridge(random_state=55)]

In [95]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor

reg = ExtraTreesRegressor(n_estimators=100, random_state=55).fit(X_train, y_train)

y_pred_train = reg.predict(X_train)
y_pred_test = reg.predict(X_test)

rmse_train= np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_val= np.sqrt(mean_squared_error(y_test, y_pred_test))

print('Train RMSE = ' + str(rmse_train))
print('Test RMSE = ' + str(rmse_val))

Train RMSE = 13.455142891638753
Test RMSE = 559.8283547677651


In [96]:
import lightgbm as lgb 
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error as mse 
from lightgbm import LGBMRegressor 
  
model = LGBMRegressor(metric='rmse') 
model.fit(X_train, y_train) 
  
# Make predictions on the training and validation data. 
y_pred_train = model.predict(X_train) 
y_pred_test = model.predict(X_test)

rmse_train= np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_val= np.sqrt(mean_squared_error(y_test, y_pred_test))

print('Train RMSE = ' + str(rmse_train))
print('Test RMSE = ' + str(rmse_val))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002803 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1029
[LightGBM] [Info] Number of data points in the train set: 32364, number of used features: 7
[LightGBM] [Info] Start training from score 3922.812909
Train RMSE = 475.7304274731118
Test RMSE = 536.8221431152924


In [97]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression


regr = RandomForestRegressor(max_depth=10, random_state=55)
regr.fit(X_train, y_train)

# Make predictions on the training and validation data. 
y_pred_train = regr.predict(X_train) 
y_pred_test = regr.predict(X_test)

rmse_train= np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_val= np.sqrt(mean_squared_error(y_test, y_pred_test))

print('Train RMSE = ' + str(rmse_train))
print('Test RMSE = ' + str(rmse_val))


Train RMSE = 471.60786090002136
Test RMSE = 566.71937656227


In [98]:
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
regu = GradientBoostingRegressor(random_state=55)
regu.fit(X_train, y_train)

y_pred_train = regu.predict(X_train) 
y_pred_test = regu.predict(X_test)

rmse_train= np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_val= np.sqrt(mean_squared_error(y_test, y_pred_test))

print('Train RMSE = ' + str(rmse_train))
print('Test RMSE = ' + str(rmse_val))

Train RMSE = 593.0881038096574
Test RMSE = 623.5109874459483


In [99]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

regressor = DecisionTreeRegressor(random_state=0)
regressor.fit(X_train, y_train)

y_pred_train = regressor.predict(X_train) 
y_pred_test = regressor.predict(X_test)

rmse_train= np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_val= np.sqrt(mean_squared_error(y_test, y_pred_test))

print('Train RMSE = ' + str(rmse_train))
print('Test RMSE = ' + str(rmse_val))

Train RMSE = 13.45484830765156
Test RMSE = 755.4775989448103


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ab417452-7889-4cb1-949d-02a1a91d883b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>